# Topological Assignment Improved - 1-most-used-qubit

In [3]:
from copy import copy

from qiskit.transpiler.basepasses import TransformationPass
from qiskit.transpiler import Layout, CouplingMap
from qiskit.circuit.library import SwapGate

from qiskit.transpiler import TranspilerError
from qiskit import QuantumCircuit, transpile
from qiskit.test.mock.backends import FakeBrooklyn
import random

from qiskit.converters import circuit_to_dag
from qiskit.dagcircuit import DAGCircuit
from qiskit.converters import dag_to_circuit

In [4]:
# Topological Assignment Improved

"""
Given the coupling map of a real device and a dag circuit, 
create a layout (map physical qubits to logical) that permits
to have less costs when a basic pass is used

"""

class TopologicalAssignmentImproved():
    
    def __init__(self,
                 coupling_map):
        
        self.coupling_map = coupling_map
        self.num_qubits = 65
        
    """
    return an ordered list of the most involved qubits in two-qubits operation
    """
    def most_involved(self, dag):
        q = {}
        for layer in dag.serial_layers():
            subdag = layer['graph']

            for gate in subdag.two_qubit_ops():
                q0 = gate.qargs[0]
                q1 = gate.qargs[1]

                if q0 in q:
                    q[q0] += 1
                else:
                    q[q0] = 1
                if q1 in q:
                    q[q1] += 1
                else:
                    q[q1] = 1

        q_list = sorted(q, key=q.get, reverse=True)
        return q_list
        
    def run(self, dag, n_most_used):
               
        # create an empty dictionary from a 65 qubit device         
        canonical_register = dag.qregs['q']
        trivial_layout = Layout.generate_trivial_layout(canonical_register)
                
        # initialize layout dictionary
        # keys --> virtual, values --> physical
        # layout_dict[virtual_qbit] to access the physical
        # list(layout_dict.values()) to retrieve the physical qubits used
        layout_dict = trivial_layout.copy().get_virtual_bits() 
        
        # initialize the dictionary such that each virtual qubits is not assigned
        # to any physical qubit
        for virtual in layout_dict.keys():
            layout_dict[virtual] = None
                
        # i: arbitrary starting qubit
        # the ideal is to start mapping the qubits from qubits 
        # which are located in the center of the circuit 
        # (see coupling_map drawing above)
        i = 31
        
        # ---- improvement ----
        # assign the n_most_used logical qubits to the central physical qubits 
        # (usually the most connected)
        most_used_qubits = self.most_involved(dag)
        for n in range(n_most_used):
            layout_dict[most_used_qubits[n]] = i
            i += 1
        
        ###
        assignments = []
        ###
        
        # walk through the dag searching for two-qubits operations
        for layer in dag.serial_layers():
            subdag = layer['graph']
            for gate in subdag.two_qubit_ops():

                physical_q0 = layout_dict[gate.qargs[0]]
                physical_q1 = layout_dict[gate.qargs[1]]
                switched = 0

                ###
                a = False
                b = False
                if physical_q0 == None:
                    a = True
                if physical_q1 == None:
                    b = True
                ###
                # gate defines a two qubits op with virtual qubits gate.qargs[0] and gate.qargs[1]
                if physical_q0 == None: #physical qubit not associated yet
                    if physical_q1 == None:
                        layout_dict[gate.qargs[0]] = i
                        physical_q0 = i
                    else:
                        physical_q0 = layout_dict[gate.qargs[1]] 
                        physical_q1 = layout_dict[gate.qargs[0]]
                        switched = 1  # keep track that now physical_q0 contains virtual_q1
 
                # physical_q0 is now assigned
                if physical_q1 == None: #physical qubit not associated yet
                    assigned = False                    
                    distance_q0 = self.coupling_map.distance_matrix[physical_q0].copy()
                    physical_assigned = [x for x in layout_dict.values() if x is not None]
                    
                    # starting from qubits with distance 1, take all the qubits that 
                    # have that distance from physical_q0 (candidates): if one of them is not assigned yet,
                    # assign it, otherwise increase the distance allowed
                    for j in range(1, int(distance_q0.max()) + 1):  #let j as close as possible to physical_q0
                        candidates = [index for index, element in enumerate(distance_q0) if element == j] #element is the distance from q0
                        random.shuffle(candidates)
                        
                        for n in candidates: #search in the nearest qubits wrt q0
                            if n not in physical_assigned:
                                layout_dict[gate.qargs[1 - switched]] = n
                                physical_q1 = n
                                assigned = True
                                break
                        if assigned == True:
                            break
                            
                    # physical_q1 should be assigned
                    if assigned == False:
                        raise TranspilerError('The layout does not match the amount of qubits in the DAG')                    
                    
                physical_assigned =  [x for x in layout_dict.values() if x is not None] 
                if len(physical_assigned) == self.num_qubits: # the mapping is completed
                    break
                    
                while i in physical_assigned:  #let i as close as possible to the previous qubits
                    i += 1
                    if i == 65:
                        i = 0
                        
                ###
                if a is True:
                    assignments.append(physical_q0 if not switched else physical_q1)
                    a = False
                if b is True:
                    assignments.append(physical_q1 if not switched else physical_q0)
                    b = False
                ###
                
        #print('\n-->This is how physical qubits have been assigned by the algorithm (in order):\n', assignments)
                
        physical = [x for x in layout_dict.values() if x is not None]
        #print('\n\nNum of physical unique qubit assigned (without unused qubits): --',len(physical), ' out of ', len(layout_dict.keys()))

        # map all the unused qubits
        used = [x for x in layout_dict.values() if x is not None]
        total = list(range(0, self.num_qubits))
        unused = [x for x in total if x not in used]  
        
        for virtual in layout_dict.keys():
            if layout_dict[virtual] == None:
                layout_dict[virtual] = unused[0]
                unused.pop(0)
        
        #check        
        physical = set(layout_dict.values())
        #print('\nNum of physical unique qubit assigned (with used qubits): --',len(physical), ' out of ', len(layout_dict.keys()))
        
        layout = Layout(layout_dict)
        return layout

In [5]:
# info device 65 qubits -- create DAG with 65 qubits
backend = FakeBrooklyn()
num_qubits =  backend.configuration().n_qubits
coupling_list = backend.configuration().coupling_map
coupling_map = CouplingMap(couplinglist=coupling_list)
device_qc = QuantumCircuit(num_qubits, num_qubits)
device_DAG = circuit_to_dag(device_qc) 

# info input circuit n <= 65 qubits -- create DAG with n qubits
file_name = 'adder-3'
input_path = './original/' + file_name + '.qasm'
qc = QuantumCircuit.from_qasm_file(path=input_path)
input_DAG = circuit_to_dag(qc) 

# build the input_DAG (n qubits) on the device_DAG (65 qubits)
# now device_DAG has 65 qubits with the structure of the input_DAG
device_DAG.compose(input_DAG)

# create and run the mapper: a mapping (layout) is returned
n_most_used = 1
tp = TopologicalAssignmentImproved(coupling_map)
tp_layout = tp.run(device_DAG, n_most_used)
print('\n--------------Mapping-------------\n---Physical qubit : Virtual qubit---\n', tp_layout)


--------------Mapping-------------
---Physical qubit : Virtual qubit---
 Layout({
33: Qubit(QuantumRegister(65, 'q'), 0),
45: Qubit(QuantumRegister(65, 'q'), 1),
30: Qubit(QuantumRegister(65, 'q'), 2),
32: Qubit(QuantumRegister(65, 'q'), 3),
39: Qubit(QuantumRegister(65, 'q'), 4),
31: Qubit(QuantumRegister(65, 'q'), 5),
0: Qubit(QuantumRegister(65, 'q'), 6),
1: Qubit(QuantumRegister(65, 'q'), 7),
2: Qubit(QuantumRegister(65, 'q'), 8),
3: Qubit(QuantumRegister(65, 'q'), 9),
4: Qubit(QuantumRegister(65, 'q'), 10),
5: Qubit(QuantumRegister(65, 'q'), 11),
6: Qubit(QuantumRegister(65, 'q'), 12),
7: Qubit(QuantumRegister(65, 'q'), 13),
8: Qubit(QuantumRegister(65, 'q'), 14),
9: Qubit(QuantumRegister(65, 'q'), 15),
10: Qubit(QuantumRegister(65, 'q'), 16),
11: Qubit(QuantumRegister(65, 'q'), 17),
12: Qubit(QuantumRegister(65, 'q'), 18),
13: Qubit(QuantumRegister(65, 'q'), 19),
14: Qubit(QuantumRegister(65, 'q'), 20),
15: Qubit(QuantumRegister(65, 'q'), 21),
16: Qubit(QuantumRegister(65, 'q'),

### Now you have the layout 
so you can use it as input of the Basic pass. n.b. You must use this modified version of the basic pass that doesn't build another trivial layout but uses the layout given as parameter.

In [6]:
from copy import copy

from qiskit.transpiler.basepasses import TransformationPass
from qiskit.transpiler import Layout
from qiskit.circuit.library import SwapGate


class BasicSwapModified(TransformationPass):
    """Maps (with minimum effort) a DAGCircuit onto a `coupling_map` adding swap gates."""

    def __init__(self,
                 coupling_map,
                 initial_layout=None):
        """Maps a DAGCircuit onto a `coupling_map` using swap gates.

        Args:
            coupling_map (CouplingMap): Directed graph represented a coupling map.
            initial_layout (Layout): initial layout of qubits in mapping
        """
        super().__init__()
        self.coupling_map = coupling_map
        self.initial_layout = initial_layout

    def run(self, dag):
        """Runs the BasicSwap pass on `dag`.

        Args:
            dag (DAGCircuit): DAG to map.

        Returns:
            DAGCircuit: A mapped DAG.

        Raises:
            TranspilerError: if the coupling map or the layout are not
            compatible with the DAG.
        """
        new_dag = DAGCircuit()
        for qreg in dag.qregs.values():
            new_dag.add_qreg(qreg)
        for creg in dag.cregs.values():
            new_dag.add_creg(creg)


        if self.initial_layout is None:
            if self.property_set["layout"]:
                self.initial_layout = self.property_set["layout"]
            else:
                self.initial_layout = Layout.generate_trivial_layout(*dag.qregs.values())
       
        if len(dag.qubits) != len(self.initial_layout):
            print('qubits on the device:--- ', len(dag.qubits), '\nqubits on the input circuit (layout):--- ',len(self.initial_layout))
            raise TranspilerError('The layout does not match the amount of qubits in the DAG')

        if len(self.coupling_map.physical_qubits) != len(self.initial_layout):
            raise TranspilerError(
                "Mappers require to have the layout to be the same size as the coupling map")

        canonical_register = dag.qregs['q']
        #trivial_layout = Layout.generate_trivial_layout(canonical_register)
        #current_layout = trivial_layout.copy()
        
        # here: use as current layout the layout given (which should be the optimized layout)
        current_layout = self.initial_layout.copy()
        for layer in dag.serial_layers():
            subdag = layer['graph']

            for gate in subdag.two_qubit_ops():
                physical_q0 = current_layout[gate.qargs[0]]
                physical_q1 = current_layout[gate.qargs[1]]
                if self.coupling_map.distance(physical_q0, physical_q1) != 1:
                    # Insert a new layer with the SWAP(s).
                    swap_layer = DAGCircuit()
                    swap_layer.add_qreg(canonical_register)

                    path = self.coupling_map.shortest_undirected_path(physical_q0, physical_q1)
                    for swap in range(len(path) - 2):
                        connected_wire_1 = path[swap]
                        connected_wire_2 = path[swap + 1]

                        qubit_1 = current_layout[connected_wire_1]
                        qubit_2 = current_layout[connected_wire_2]

                        # create the swap operation
                        swap_layer.apply_operation_back(SwapGate(),
                                                        qargs=[qubit_1, qubit_2],
                                                        cargs=[])

                    # layer insertion
                    order = current_layout.reorder_bits(new_dag.qubits)
                    new_dag.compose(swap_layer, qubits=order)

                    # update current_layout
                    for swap in range(len(path) - 2):
                        current_layout.swap(path[swap], path[swap + 1])

            order = current_layout.reorder_bits(new_dag.qubits)
            new_dag.compose(subdag, qubits=order)

        return new_dag

In [7]:
# test for the file comment
physical_list = list(tp_layout.get_physical_bits().keys())
physical_str = ''
for elem in physical_list:
    physical_str = physical_str + str(elem) + ' '

print(physical_str)

33 45 30 32 39 31 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 34 35 36 37 38 40 41 42 43 44 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 


### From the input
run the mapper and produce a .qasm file

In [8]:
out_dag_path = './output/DAG/'+ file_name + '.png'
out_qasm_path = './output/qasm/' + file_name + '.qasm'

# apply the swap and generate the DAG drawing - VERY EXPENSIVE, DON'T RUN IF NOT NECESSARY
basic_pass = BasicSwapModified(coupling_map, tp_layout)
out_DAG = basic_pass.run(device_DAG)
#out_DAG.draw(filename=out_dag_path)

# generate the .qasm
out_qc = dag_to_circuit(out_DAG)
out_comment = '// i ' + physical_str + '\n\n'
out_string = out_comment + out_qc.qasm()
#print(out_string)

In [9]:
#open text file
text_file = open(out_qasm_path, "w")
 
#write string to file
text_file.write(out_string)
 
#close file
text_file.close()

### Check

In [10]:
from mqt import qcec

result = qcec.verify(input_path, out_qasm_path)
print(result)

{
  "circuit1": {
    "n_gates": 71,
    "n_qubits": 6,
    "name": "adder-3"
  },
  "circuit2": {
    "n_gates": 82,
    "n_qubits": 65,
    "name": "adder-3"
  },
  "equivalence": "equivalent",
  "statistics": {
    "max_nodes": 150,
    "method": "G -> I <- G'",
    "preprocessing_time": 0.0138523,
    "strategy": "proportional",
    "verification_time": 0.0162315
  }
}
